In [ ]:
pip install pytesseract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
pip install datasets rouge-score nltk tensorboard py7zr --upgrade


                                              0.0/474.6 kB ? eta -:--:--
     ----------------------------           358.4/474.6 kB 7.4 MB/s eta 0:00:01
     -------------------------------------- 474.6/474.6 kB 9.9 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
                                              0.0/1.5 MB ? eta -:--:--
     -----                                    0.2/1.5 MB 6.7 MB/s eta 0:00:01
     ------------                             0.5/1.5 MB 7.5 MB/s eta 0:00:01
     ----------------                         0.6/1.5 MB 4.8 MB/s eta 0:00:01
     -----------------                        0.7/1.5 MB 5.2 MB/s eta 0:00:01
     ------------------------------           1.2/1.5

In [2]:
pip uninstall -y transformers accelerate

Found existing installation: transformers 4.29.1
Uninstalling transformers-4.29.1:
  Successfully uninstalled transformers-4.29.1
Note: you may need to restart the kernel to use updated packages.


# New Section

In [5]:
pip install evaluate

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade accelerate


                                              0.0/219.1 kB ? eta -:--:--
     ----------------------------           163.8/219.1 kB 3.3 MB/s eta 0:00:01
     -------------------------------------- 219.1/219.1 kB 3.3 MB/s eta 0:00:00


In [6]:
pip install transformers==4.28.0

                                              0.0/7.0 MB ? eta -:--:--
     -                                        0.3/7.0 MB 5.2 MB/s eta 0:00:02
     ---                                      0.6/7.0 MB 6.8 MB/s eta 0:00:01
     -----                                    0.9/7.0 MB 6.4 MB/s eta 0:00:01
     -------                                  1.3/7.0 MB 7.0 MB/s eta 0:00:01
     ------------                             2.2/7.0 MB 9.3 MB/s eta 0:00:01
     -----------------                        3.0/7.0 MB 11.2 MB/s eta 0:00:01
     --------------------                     3.5/7.0 MB 11.2 MB/s eta 0:00:01
     -----------------------                  4.1/7.0 MB 11.4 MB/s eta 0:00:01
     ---------------------------              4.7/7.0 MB 11.6 MB/s eta 0:00:01
     ------------------------------           5.4/7.0 MB 11.8 MB/s eta 0:00:01
     ----------------------------------       5.9/7.0 MB 11.8 MB/s eta 0:00:01
     -------------------------------------    6.5/7.0 MB 12.1 MB/

In [7]:
from huggingface_hub import notebook_login

notebook_login()


In [9]:
import pandas as pd

# Define the path to the CSV file
data_path = "edos_labelled_aggregated.csv"

# Load the dataset using pandas
dataset = pd.read_csv(data_path)

from datasets import load_dataset

## Load dataset from the hub
#dataset = load_dataset(data.csv)

print(f"Train dataset size: {len(dataset[dataset.split=='train'])}")
print(f"Test dataset size: {len(dataset[dataset.split=='test'])}")

# Train dataset size: 14732
# Test dataset size: 819


Train dataset size: 14000
Test dataset size: 4000


In [10]:
print('a')
from datasets import concatenate_datasets
from datasets import Dataset
train_df= Dataset.from_pandas(dataset[dataset.split=='train'])
test_df= Dataset.from_pandas(dataset[dataset.split=='test'])
dataset2=Dataset.from_pandas(dataset)

a


In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)



C:\Users\ilaun\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ilaun\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [12]:
from datasets import concatenate_datasets
from datasets import Dataset

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([train_df, test_df]).map(lambda x: tokenizer(x["text"], truncation=True), batched=True, remove_columns=["text", "label_sexist"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([train_df, test_df]).map(lambda x: tokenizer(x["text"], truncation=True), batched=True, remove_columns=["text", "label_sexist"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")


Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Max source length: 124


Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Max target length: 124


In [13]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["Classify " + item for item in sample["text"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["label_sexist"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset2.map(preprocess_function, batched=True, remove_columns=["rewire_id", "text", "label_sexist", "label_category", "label_vector"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset.features)}")


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['split', 'input_ids', 'attention_mask', 'labels']


In [14]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="google/flan-t5-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)


In [15]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ilaun\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [16]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)



In [17]:
pip install accelerate

Note: you may need to restart the kernel to use updated packages.


In [18]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-{'edos_labelled_aggregated'}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset.filter(lambda example: example["split"] == "train"),
    eval_dataset=tokenized_dataset.filter(lambda example: example["split"] == "test"),
    compute_metrics=compute_metrics,
)


Filter:   0%|          | 0/20000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [19]:
# Start training
trainer.train()


C:\Users\ilaun\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.080300,0.075009,95.575000,72.100000,95.575000,95.583300,4.817250
2,0.067900,0.071489,95.533300,69.250000,95.541700,95.550000,4.762000
3,0.058900,0.074603,95.783300,70.125000,95.791700,95.791700,4.771500
4,0.050800,0.087677,95.775000,69.750000,95.775000,95.783300,4.764250
5,0.047400,0.096577,95.650000,69.625000,95.666700,95.675000,4.765250


TrainOutput(global_step=8750, training_loss=0.06899046303885324, metrics={'train_runtime': 99527.1798, 'train_samples_per_second': 0.703, 'train_steps_per_second': 0.088, 'total_flos': 1.198325366784e+16, 'train_loss': 0.06899046303885324, 'epoch': 5.0})